# COMS 4995_002 Deep Learning Assignment 1
Due on Monday, Oct 9, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Pulkit Jain, pj2313

Member 2: Greg Kocher, gk2500

Member 3: Ratheet Pandya, rp2707

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [405]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {
            'layerDimensions' : layer_dimensions
        }
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        self.iterations_finished = 0
        self.weights = []
        self.biases = []
        
                
        """
        other ideas for increased performance:
        - TA's suggested batchnorm layers
        
        Some that seem easy:
        - decaying learning rate (function of iteration number)
        - random crops/reflections[left-right only]/distortions of input data
        - other random initializations [Xavier, MSRA] - probably not have 
          much effect since we aren't using very deep network??
        
        - #self.reg_p_norm = 2 #If doing Lp norm regularization
        - other activation functions [ELU]
        """
        
        # init parameters

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        Z = np.matmul(W, A) + b
        return Z
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        return self.relu(A)


    def relu(self, X):
        A = np.maximum(0, X)
        return A

            
    def dropout(self, A, prob):
        """
        :param A: 
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        rng = np.random.RandomState(123)
        M = rng.binomial(size=A.shape,
                            n=1,
                            p=1-prob)
        A *= M
        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        A = X
        

        cache = {
            'biases' : [],
            'weights' : [],
            'dropoutMasks' : [],
            'affines' : [],
            'activations' : [A]
        }
        
        #After already done 1 iteration, reuse the weights and biases we learned:
        if self.iterations_finished > 0:
            cache['biases'] = self.biases
            cache['weights'] = self.weights
            #print(len(cache['weights']))
            #print(cache['weights'][0].shape)
            #print(cache['weights'][1].shape)
            #print(cache['weights'][2].shape)
        
        #same for below random initialization of W and b, are they being overwritten?
        
        for layer in range(1,self.num_layers):
            #On 1st training iteration only, randomly initialize weights and biases:
            if self.iterations_finished==0:
                W = np.random.normal(0., .1, size=(self.parameters['layerDimensions'][layer], self.parameters['layerDimensions'][layer - 1]))
                b = np.zeros(shape=(self.parameters['layerDimensions'][layer], 1))
            else:
                W = cache['weights'][layer-1]
                b = cache['biases'][layer-1]
            Z= self.affineForward(A, W, b)
            
            #So relu not applied to last layer:
            A = self.activationForward(Z) if layer==self.num_layers-1 else Z
            
            if self.drop_prob > 0:
                A, M = self.dropout(A,self.drop_prob)
                cache['dropoutMasks'].append(M)
            cache['affines'].append(Z)
            cache['activations'].append(A)
            
            if self.iterations_finished==0:
                cache['weights'].append(W)
                cache['biases'].append(b)    

        AL = A
        return AL, cache
    
    def costFunction(self, AL, y, cache):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        
        epsilon = 10e-9 #Since taking logs of very small numbers is giving infs
        S = y.size
        
        #Get softmax cost per sample:
        #(both numerators and denominators are length S vectors)
        #denominators = np.exp(AL).sum(axis=0)
        #numerators = np.array([np.exp(AL[y[i]-1,i]) for i in range(S)])#-1 is assuming class labels start at 1. If they start at 0, get rid of -1.
        #Actually will end up needing for gradients so just do as below...
        
        #print(AL)
        #print(AL.min())
        #print(AL.max())
        
        #print(np.exp(AL))
        #print(np.exp(AL).min())
        #print(np.exp(AL).max())
        
        softmax_out = np.exp(AL)/(np.exp(AL).sum(axis=0)+epsilon)
        #print(softmax_out)
        #print(softmax_out.min())
        #print(softmax_out.max())

        #Now get the cross-entropy / log-loss of this softmax:
        #cost is the average over all samples
        cost = (-1./S)*np.nansum(np.log(softmax_out))#+epsilon)) #Should deal with nans, but for now just use nansum...

        
        #Also get the regularization cost:
        if self.reg_lambda > 0.:
            # add regularization
            #If want to do other Lp norms as easy bonus:
            #in init: self.reg_p_norm = 2#int p for p-norm regularization
            #FOr now just using L2 norm:
            #||W||^2
            reg_cost = self.reg_lambda*np.array([np.sum(i.flatten()**2) for i in cache['weights']]).sum()
            #print(reg_cost)
            #print(cost)
            cost += reg_cost
        
        
        
        # gradient of cost
        #gradient through cross entropy, then sigmoid, turns out to be:
        #for a single trainign example:
        #dAL_i = sigmoid(z_i) - I(class==i);
        #when go over all training batch examples, 
        #take average over axis 1. Is just sigmoid out array, with certain elements -1.
        #Those elements are the ones corresponding to correct class label, for a given
        #training example:
        softmax_out[y,np.arange(S)] -= 1.
        #Sum over axis 1 and then scale by S, i.e. take mean over samples:
        #dAL = softmax_out.mean(axis=1)#actually since using 2D dropout mask,
        #don't do the averaging now. Leave as 2D matrix with axis1 size = S, the number of samples
        dAL = softmax_out
        
        #Since we don't use ReLU on last layer and treat it 
        #manually in this function by feeding it through softmax layer,
        #get rid of last layer activation here since we basically bypass 
        #the gradient calculation for this last layer and do it separately.
        #THen in the layers before this last one, can treat as usual.
        _ = cache['activations'].pop()
        #Same thing for dropout mask, if doing dropout:
        if self.drop_prob > 0.:
            _ = cache['dropoutMasks'].pop()
        
        return cost, dAL

    
    def affineBackward(self, dA_prev, cache, layer):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :layer : since you need to multiply by weights/biases
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        
        #print('layer',layer)
        
        S = cache['activations'][0].shape[1]
        
        #Use the derivative from the later layer
        #["previous" since goign in reverse]:
        #dA_prev
        
        W = cache['weights'][layer-1]
        dA = np.dot(W.T,dA_prev)
        #print(dA.shape)

        #If doing dropout:
        if self.drop_prob > 0.:
            #Sropout not used directly on input layer, even if mask is generated for it.
            if layer > 1:
                dA = self.dropout_backward(dA, cache)
        dA = self.activationBackward(dA, cache, None, activation="relu")
        
        #Repeat dA_prev since have S samples:
        #dW = np.dot(np.repeat(dA_prev.reshape((dA_prev.size,1)),S,axis=1),cache['activations'][-1].T)
        #Since dropout mask, just keep dA_prev in 2D [don't average over samples]
        dW = np.dot(dA_prev,cache['activations'][-1].T)
        #print(dW)
        #print(dW.min(),dW.max())
        #print(dW.shape)#(10,100)

        
        #If regularized, also add in gradients from reg cost:
        if self.reg_lambda > 0:
            dW += self.reg_lambda*W #For now assuming only L2 reg.
        
        #d_output / db = I(if i)
        db = np.ones(cache['biases'][layer-1].shape)
        
        
        #Now that done using the activations, get rid of them 
        #so indexing correct for next iterations:
        cache['activations'].pop()
        #print(len(cache['activations']))
        #print(cache['activations'][0].shape)

        return dA, dW, db

    
    
    def activationBackward(self, dA, cache, layer, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        cached_x = cache['activations'][-1]
        return self.relu_derivative(dA, cached_x)

        
    def relu_derivative(self, dx, cached_x):

        #S = cached_x.shape[1]
        #Repeat dx since have S samples:
        #dx = np.repeat(dx.reshape((dx.size,1)),S,axis=1)
        
        #In positive region, dx=1; in negative region = 0:
        dx = np.ones(cached_x.shape)*dx
        #Mask the negative region to 0.
        dx[cached_x <= 0.] = 0.
        #Could treat the exact ==0. case differently but to precision will never get ==0.
        
        #Average over axis 1 [averaged over samples]
        #dx = dx.mean(axis=1)#No longer do this since dropout mask
        return dx

    
    def dropout_backward(self, dA, cache):
        dA *= cache['dropoutMasks'].pop()
        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {
            'dW' : [],
            'db' : []
         }
        
        
        dA = dAL
        for layer in range(self.num_layers-1):
            dA, dW, db = self.affineBackward(dA,cache,self.num_layers-layer-1)
            gradients['dW'].append(dW) 
            gradients['db'].append(db) 
            
            #Was easier to just put inside affineBackward...
            #if self.drop_prob > 0:
            #    #If not working on the input layer (don't use dropout on it):
            #    if layer < self.num_layers-2 #-2 since: 
            #        self.dropout_backward(dA,cache)
           
        # TODO    
        #if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            #GK: I'm just adding them to dAL in the costfunction so should be ok w/o this here
        gradients['dW'].reverse()
        gradients['db'].reverse() 
        return gradients


    def updateParameters(self, gradients, alpha, cache):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        
        cache: to update the parameters we are learning [weights and biases],
        those values also need to be passed in.
        """
        
        #Weights
        #print(cache['weights'][0])
        #print(gradients['dW'][0].min())
        #print(gradients['dW'][0].max())
        cache['weights'] = [cache['weights'][i]  - alpha*gradients['dW'][i] for i in range(len(cache['weights']))] 
        #print(cache['weights'][0])
        #print(cache['weights'][0].max())
        #print(cache['weights'][0].min())

        #biases
        #print(cache['biases'][0])
        cache['biases'] = [cache['biases'][i]  - alpha*gradients['db'][i] for i in range(len(cache['biases']))] 
        #print(cache['biases'][0])
    
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        
        AL, _ = self.forwardPropagation(X)
        
        #Get probability per class per sample, then take argmax:
        #Technically don't need to go through softmax calculations, etc. since max here is max after too.
        #SO just get argmax for each sample:
        max_classes = np.argmax(AL,axis=0)
        
        y_pred = list(max_classes)
        print(y_pred)
        print(np.bincount(max_classes))
        #y_pred = [100, 200, 300]
        
        return y_pred

    
    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        selector = np.random.choice(np.arange(np.size(y)), batch_size, replace=False)
        return X[:, selector], y[selector]

    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        for i in range(0, iters):
            # get minibatch
            X_batch, y_batch = self.get_batch(X, y, batch_size)
            
            # forward prop
            AL, cache = self.forwardPropagation(X_batch)

            # compute loss
            cost, dAL = self.costFunction(AL, y_batch, cache)
            #print('cost',cost)
            
            # TODO compute gradients
            gradients = self.backPropagation(dAL, y_batch, cache)
            #print('gradients',gradients,len(gradients['dW']),gradients['dW'][0].shape)

            # TODO update weights and biases based on gradient
            #print('alpha',alpha)
            self.updateParameters(gradients, alpha, cache) #will update weights + biases in place

            
            #Store the weights and biases that were just updated: 
            self.weights = cache['weights']
            self.biases = cache['biases']
            #print(len(self.weights))
            #print(len(self.biases))
            #print(self.weights[0].shape)
            #print(self.biases[0].shape)
            
            
            #Just some counter which is only used to make sure this isn't the first iteration
            #since on 1st iteration only we randomly initialize params.
            self.iterations_finished += 1
            print(self.iterations_finished)
            print('\n')
            
            if i % print_every == 0:
                # TODO compute + print cost, train and validation set accuracies
                y_batch_predicted = self.predict(X_batch)
                train_accuracy = y_batch_predicted
                validation_accuracy = 0.0
                print('cost: {0}, train accuracy: {1}, validation accuracy: {2}'.format(
                    cost, train_accuracy, validation_accuracy))

In [28]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [35]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:#[:20000]:#!!!!!!! changed for memory issues, but change this back to "files:"
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [36]:
# Load the data
data_root_path = '/Users/ratheet/coms4995-deeplearning/HW1/git/cifar10-hw1/'
data_root_path = '.././cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [406]:
layer_dimensions = [X_train.shape[0], 100, 50, 10]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=10, alpha=0.1, batch_size=16, print_every=10)#change back batch_size=10 #!=10 to debug not square

1


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[16]
cost: 43.803813332570854, train accuracy: 0.0, validation accuracy: 0.0
2


3


4


5


6


7


8


9


10




C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: overflow encountered in exp
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:177: RuntimeWarning: divide by zero encountered in log
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in maximum
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:296: RuntimeWarning: invalid value encountered in less_equal


In [29]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [30]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

(3,)


array([100, 200, 300])

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [407]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=.25, reg_lambda=.01)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

1


[6, 0, 9, 5, 0, 0, 9, 0, 0, 0, 5, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 2, 6, 0, 0, 4, 0, 9, 6, 0, 0, 9, 0, 0, 0, 0, 9, 4, 6, 0, 9, 0, 0, 0, 2, 0, 0, 3, 4, 0, 0, 5, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 4, 8, 9, 6, 0, 0, 0, 9, 0, 6, 0, 0, 5, 0, 0, 2, 0, 2, 0, 7, 3, 3, 2, 0, 0, 9, 6, 5, 0, 3, 0, 0, 8, 9, 0, 0, 0, 0, 0, 6, 0, 8, 4, 0, 6, 0, 0, 7, 8, 2, 0, 0, 9, 0, 8, 8, 5, 3, 0, 0, 2, 2, 0, 9, 2, 0, 0, 0, 9, 6, 7, 6, 0, 0, 0, 0, 5, 0, 8, 0, 0, 9, 0, 0, 0, 2, 0, 5, 0, 0, 0, 0, 0, 9, 0, 0, 0, 8, 0, 0, 0, 9, 0, 6, 3, 0, 5, 0, 8, 4, 6, 0, 0, 0, 0, 0, 0, 9, 0, 5, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 9, 9, 0, 5, 0, 0, 0, 8, 0, 2, 9, 9, 9, 6, 0, 0, 6, 5, 0, 4, 0, 6, 0, 0, 3, 6, 0, 4, 6, 9, 8, 0, 9, 0, 2, 0, 0, 0, 6, 5, 5, 0, 0, 5, 2, 0, 7, 7, 6, 0, 0, 0, 3, 0, 5, 3, 0, 0, 0, 0, 5, 5, 0, 1, 0, 0, 0, 0, 9, 0, 0, 0, 0, 2, 0, 0, 0, 0, 9, 0, 0, 6, 0, 0, 9, 0, 9, 0, 9, 2, 0, 0, 2, 1, 0, 9, 9, 9, 3, 4, 0, 8, 8, 6, 6, 2, 6, 0, 0, 9, 7, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 9, 0, 0, 0, 0, 5, 0, 0, 0, 0, 6, 2, 7, 6, 4,

23


24


25


26


27


28


29


30


31


[1, 1, 9, 4, 3, 0, 2, 0, 8, 1, 9, 8, 0, 6, 1, 2, 6, 9, 0, 9, 7, 2, 5, 7, 7, 9, 5, 9, 6, 0, 7, 9, 1, 0, 0, 8, 2, 4, 4, 3, 9, 1, 0, 2, 2, 0, 8, 5, 4, 1, 4, 5, 0, 6, 2, 4, 0, 2, 6, 0, 3, 3, 0, 9, 4, 1, 9, 2, 0, 2, 5, 8, 0, 0, 0, 5, 9, 0, 8, 2, 1, 2, 7, 7, 2, 3, 2, 2, 4, 9, 1, 5, 7, 7, 6, 0, 6, 9, 4, 1, 2, 0, 4, 6, 6, 8, 4, 0, 3, 9, 2, 4, 4, 2, 1, 6, 2, 3, 8, 8, 2, 3, 2, 9, 2, 2, 1, 5, 5, 4, 8, 0, 1, 6, 1, 6, 0, 0, 4, 0, 6, 0, 8, 0, 6, 0, 9, 0, 7, 5, 0, 5, 0, 8, 1, 7, 0, 8, 6, 8, 7, 8, 2, 7, 0, 2, 5, 2, 3, 4, 8, 1, 8, 1, 1, 9, 3, 4, 0, 0, 6, 9, 3, 7, 0, 8, 0, 2, 2, 7, 0, 8, 0, 0, 4, 9, 2, 5, 2, 7, 0, 7, 2, 3, 5, 2, 4, 6, 6, 7, 3, 5, 8, 3, 4, 6, 3, 8, 3, 4, 0, 4, 9, 4, 5, 0, 8, 8, 2, 7, 0, 5, 1, 5, 2, 7, 4, 5, 8, 2, 2, 7, 6, 2, 5, 6, 3, 4, 7, 3, 2, 1, 1, 0, 1, 5, 6, 1, 5, 1, 2, 0, 6, 7, 2, 0, 0, 1, 1, 7, 2, 4, 1, 1, 6, 8, 7, 6, 4, 0, 6, 8, 9, 2, 7, 0, 8, 1, 7, 4, 9, 9, 7, 6, 2, 8, 8, 6, 5, 2, 8, 5, 3, 9, 7, 1, 2, 7, 0, 0, 0, 1, 0, 0, 0, 8, 2, 7, 

52


53


54


55


56


57


58


59


60


61


[1, 1, 9, 4, 8, 0, 8, 1, 8, 9, 9, 7, 0, 3, 1, 3, 0, 9, 7, 7, 9, 3, 5, 1, 4, 4, 8, 9, 9, 0, 5, 5, 9, 1, 8, 8, 6, 4, 7, 1, 4, 0, 0, 1, 7, 0, 7, 3, 4, 1, 8, 6, 7, 3, 2, 9, 9, 2, 6, 0, 5, 0, 2, 9, 6, 8, 9, 6, 2, 7, 4, 2, 0, 6, 0, 5, 5, 0, 8, 2, 4, 2, 9, 7, 8, 3, 2, 6, 0, 4, 1, 4, 4, 8, 6, 0, 6, 9, 4, 1, 2, 0, 5, 1, 6, 8, 1, 7, 3, 9, 2, 7, 6, 8, 1, 5, 9, 4, 2, 8, 2, 3, 2, 9, 2, 8, 8, 3, 5, 0, 8, 4, 9, 6, 7, 6, 5, 0, 8, 8, 6, 3, 8, 1, 6, 5, 2, 8, 7, 5, 3, 5, 5, 5, 3, 7, 0, 9, 4, 8, 3, 8, 1, 7, 6, 6, 9, 2, 3, 8, 8, 0, 1, 5, 1, 4, 3, 2, 0, 0, 8, 9, 1, 5, 9, 1, 1, 2, 5, 7, 0, 4, 4, 7, 8, 9, 2, 7, 2, 7, 2, 4, 2, 4, 3, 9, 9, 6, 6, 7, 8, 5, 0, 3, 8, 6, 4, 4, 4, 5, 0, 4, 5, 8, 8, 0, 8, 7, 7, 3, 6, 5, 2, 5, 3, 7, 4, 5, 2, 6, 7, 7, 4, 1, 3, 6, 8, 0, 7, 6, 7, 8, 7, 9, 5, 5, 5, 2, 4, 6, 9, 0, 8, 6, 1, 1, 0, 2, 7, 7, 2, 8, 9, 6, 6, 9, 8, 6, 2, 0, 5, 2, 9, 8, 6, 0, 8, 1, 2, 1, 9, 9, 8, 1, 2, 8, 8, 7, 6, 2, 7, 4, 7, 9, 5, 9, 9, 4, 9, 0, 8, 0, 1, 0, 8, 8, 2

82


83


84


85


86


87


88


89


90


91


[6, 0, 9, 1, 2, 5, 1, 0, 4, 9, 9, 6, 1, 3, 1, 3, 8, 8, 1, 7, 7, 6, 5, 5, 7, 9, 1, 9, 2, 0, 5, 9, 6, 0, 0, 8, 6, 7, 4, 0, 1, 0, 3, 2, 7, 0, 8, 5, 1, 2, 3, 6, 7, 7, 7, 5, 4, 2, 6, 1, 0, 0, 7, 9, 4, 1, 9, 7, 2, 6, 5, 4, 8, 0, 5, 5, 5, 8, 8, 2, 9, 8, 0, 7, 2, 5, 2, 3, 5, 9, 1, 4, 7, 5, 6, 5, 8, 9, 4, 1, 2, 3, 0, 6, 0, 8, 3, 6, 7, 9, 7, 5, 3, 1, 1, 8, 9, 7, 2, 8, 5, 3, 2, 5, 2, 8, 0, 5, 2, 5, 2, 4, 9, 2, 6, 2, 6, 9, 6, 7, 7, 0, 8, 5, 0, 5, 0, 8, 7, 5, 0, 2, 0, 9, 3, 6, 7, 9, 5, 8, 3, 3, 3, 1, 6, 6, 5, 2, 3, 4, 8, 1, 8, 2, 1, 1, 1, 8, 1, 1, 2, 8, 1, 5, 0, 9, 4, 2, 5, 7, 6, 8, 3, 0, 8, 9, 0, 2, 8, 7, 3, 4, 2, 3, 3, 9, 9, 0, 6, 7, 8, 4, 8, 3, 8, 6, 7, 3, 9, 2, 0, 2, 6, 9, 4, 0, 3, 2, 2, 7, 6, 7, 2, 5, 2, 6, 4, 5, 1, 6, 7, 7, 4, 1, 5, 6, 3, 0, 5, 9, 7, 8, 1, 2, 5, 1, 5, 1, 6, 6, 9, 0, 6, 1, 2, 0, 0, 1, 7, 2, 5, 8, 3, 2, 6, 8, 6, 3, 9, 0, 9, 5, 9, 8, 6, 7, 3, 1, 6, 5, 6, 9, 4, 6, 1, 8, 8, 7, 1, 7, 4, 4, 0, 3, 5, 7, 9, 4, 1, 1, 8, 1, 5, 1, 0, 7, 0

112


113


114


115


116


117


118


119


120


121


[5, 0, 9, 1, 2, 5, 2, 0, 4, 9, 4, 6, 1, 3, 1, 3, 8, 9, 7, 7, 9, 4, 5, 2, 7, 4, 0, 9, 9, 0, 5, 9, 5, 0, 8, 8, 6, 4, 7, 5, 9, 0, 3, 2, 2, 0, 6, 5, 1, 2, 5, 6, 8, 0, 2, 5, 5, 0, 6, 0, 0, 8, 2, 2, 4, 0, 5, 2, 2, 2, 1, 8, 7, 6, 7, 8, 0, 2, 8, 2, 4, 2, 9, 1, 8, 5, 0, 7, 0, 9, 6, 5, 0, 2, 6, 0, 6, 9, 1, 1, 2, 1, 5, 6, 0, 8, 5, 8, 3, 2, 2, 7, 1, 2, 3, 5, 9, 2, 4, 8, 2, 3, 9, 9, 2, 2, 1, 5, 2, 0, 4, 0, 1, 6, 3, 6, 9, 1, 0, 7, 6, 8, 8, 7, 6, 5, 0, 8, 8, 5, 6, 7, 0, 9, 3, 0, 6, 6, 5, 8, 3, 3, 1, 7, 0, 8, 1, 2, 1, 0, 8, 6, 7, 4, 6, 1, 1, 6, 2, 0, 1, 9, 3, 5, 0, 8, 1, 2, 5, 3, 6, 8, 0, 7, 4, 9, 0, 5, 2, 1, 0, 4, 3, 1, 7, 2, 9, 6, 6, 3, 1, 5, 0, 6, 4, 6, 3, 4, 3, 2, 7, 5, 2, 9, 8, 7, 4, 5, 2, 6, 0, 5, 3, 2, 2, 7, 1, 5, 2, 2, 2, 7, 3, 1, 3, 6, 5, 0, 7, 3, 2, 1, 0, 4, 2, 5, 5, 1, 4, 6, 9, 0, 4, 6, 2, 7, 7, 2, 3, 2, 1, 5, 5, 1, 6, 8, 2, 4, 2, 0, 4, 5, 9, 5, 3, 5, 3, 1, 6, 4, 4, 9, 7, 5, 6, 8, 8, 4, 0, 2, 8, 5, 3, 9, 5, 1, 4, 5, 0, 7, 9, 1, 1, 

...

862


863


864


865


866


867


868


869


870


871


[1, 7, 8, 4, 7, 4, 8, 2, 5, 0, 8, 9, 3, 8, 1, 8, 5, 2, 1, 8, 0, 3, 2, 8, 9, 4, 1, 4, 6, 0, 5, 9, 1, 9, 1, 7, 2, 6, 3, 9, 3, 9, 7, 1, 3, 7, 7, 3, 0, 0, 4, 2, 8, 5, 5, 5, 1, 2, 1, 9, 3, 0, 2, 0, 2, 5, 9, 1, 9, 0, 7, 8, 0, 6, 9, 5, 1, 5, 8, 7, 0, 6, 2, 9, 8, 9, 8, 7, 8, 2, 6, 3, 4, 6, 5, 5, 4, 8, 0, 7, 2, 5, 3, 1, 6, 4, 7, 8, 3, 2, 8, 8, 3, 5, 9, 8, 6, 6, 8, 5, 9, 3, 5, 3, 1, 2, 8, 0, 3, 5, 9, 3, 5, 3, 1, 3, 9, 1, 6, 8, 5, 3, 4, 4, 6, 9, 8, 5, 8, 8, 8, 7, 5, 0, 3, 5, 4, 6, 8, 8, 9, 3, 9, 5, 5, 2, 8, 6, 9, 3, 9, 1, 5, 1, 1, 9, 8, 2, 5, 8, 9, 3, 7, 2, 0, 3, 1, 7, 8, 5, 2, 2, 1, 3, 8, 0, 5, 5, 6, 7, 0, 5, 2, 3, 6, 1, 7, 5, 7, 7, 3, 5, 0, 8, 4, 6, 2, 5, 9, 5, 0, 6, 0, 9, 1, 1, 0, 9, 4, 0, 8, 5, 9, 6, 5, 7, 1, 7, 0, 0, 9, 7, 2, 3, 6, 6, 3, 4, 5, 6, 3, 6, 0, 9, 1, 1, 5, 8, 7, 6, 5, 8, 4, 6, 0, 1, 2, 1, 8, 0, 1, 5, 9, 3, 0, 6, 7, 6, 6, 9, 5, 6, 1, 8, 1, 6, 8, 1, 8, 5, 9, 6, 0, 4, 0, 7, 6, 7, 0, 5, 7, 4, 5, 9, 5, 7, 7, 4, 7, 2, 9, 0, 5, 

893


894


895


896


897


898


899


900


901


[5, 0, 3, 7, 1, 0, 1, 2, 5, 0, 4, 7, 9, 3, 1, 8, 8, 9, 7, 6, 7, 6, 5, 0, 6, 9, 3, 9, 9, 0, 6, 9, 1, 0, 4, 8, 6, 4, 6, 5, 0, 1, 3, 2, 5, 1, 9, 3, 0, 2, 1, 6, 7, 2, 0, 0, 6, 2, 6, 5, 6, 5, 2, 5, 2, 8, 6, 2, 2, 0, 5, 9, 4, 6, 4, 4, 1, 8, 3, 2, 9, 1, 0, 0, 1, 7, 5, 4, 8, 6, 0, 1, 2, 8, 5, 0, 5, 5, 0, 0, 2, 1, 7, 6, 1, 2, 1, 0, 2, 2, 8, 6, 0, 1, 2, 5, 9, 4, 8, 5, 5, 5, 5, 5, 3, 8, 6, 7, 9, 1, 0, 0, 9, 1, 3, 3, 9, 4, 4, 6, 0, 8, 4, 5, 0, 8, 3, 8, 0, 6, 2, 7, 7, 7, 6, 2, 7, 9, 0, 2, 4, 4, 5, 9, 5, 6, 2, 1, 6, 6, 9, 1, 7, 1, 3, 6, 0, 8, 5, 5, 9, 3, 0, 5, 6, 8, 3, 0, 6, 1, 0, 8, 8, 0, 6, 9, 0, 0, 5, 5, 0, 4, 2, 8, 9, 1, 8, 1, 6, 5, 4, 2, 1, 1, 4, 6, 4, 3, 9, 7, 3, 4, 2, 7, 4, 0, 4, 8, 2, 4, 3, 4, 9, 8, 5, 6, 0, 5, 3, 1, 8, 9, 1, 1, 4, 1, 8, 6, 9, 1, 9, 5, 6, 3, 9, 7, 8, 1, 6, 2, 6, 3, 4, 1, 6, 1, 0, 8, 3, 2, 5, 0, 0, 6, 0, 2, 5, 8, 2, 0, 9, 6, 2, 2, 9, 1, 6, 1, 2, 4, 8, 2, 8, 2, 2, 7, 8, 3, 8, 2, 7, 4, 5, 3, 4, 2, 9, 4, 3, 2, 5, 1, 5, 7, 9, 

922


923


924


925


926


927


928


929


930


931


[8, 9, 3, 5, 5, 9, 1, 9, 2, 1, 0, 9, 3, 0, 6, 8, 7, 8, 5, 3, 4, 6, 6, 6, 0, 4, 2, 7, 2, 6, 1, 9, 1, 0, 9, 9, 1, 5, 6, 9, 9, 4, 7, 1, 9, 0, 5, 3, 0, 0, 4, 6, 8, 5, 5, 5, 9, 3, 6, 8, 0, 9, 1, 8, 9, 3, 4, 6, 2, 5, 8, 4, 8, 6, 0, 6, 9, 1, 3, 0, 0, 6, 2, 0, 8, 3, 4, 5, 0, 5, 3, 3, 6, 9, 0, 6, 0, 9, 4, 8, 6, 4, 0, 1, 1, 4, 9, 0, 3, 9, 0, 8, 3, 2, 4, 8, 6, 0, 4, 6, 9, 2, 5, 3, 1, 3, 6, 7, 5, 1, 9, 0, 5, 6, 3, 2, 0, 3, 8, 6, 6, 3, 8, 1, 2, 8, 2, 5, 8, 6, 0, 1, 1, 8, 8, 0, 1, 8, 5, 3, 3, 3, 1, 0, 0, 2, 2, 5, 9, 3, 5, 0, 7, 7, 3, 1, 3, 6, 5, 7, 8, 3, 7, 7, 9, 8, 4, 0, 6, 5, 0, 2, 8, 9, 1, 8, 0, 0, 8, 5, 9, 1, 3, 2, 3, 0, 4, 0, 3, 3, 3, 5, 6, 4, 6, 6, 3, 4, 9, 2, 5, 9, 1, 1, 8, 0, 0, 9, 2, 6, 8, 0, 9, 5, 5, 7, 0, 6, 9, 3, 2, 3, 2, 1, 2, 5, 3, 9, 5, 6, 0, 2, 1, 3, 7, 9, 1, 1, 5, 0, 6, 3, 4, 1, 6, 0, 9, 6, 8, 2, 5, 9, 1, 0, 0, 2, 4, 6, 9, 5, 7, 6, 9, 2, 3, 6, 3, 3, 2, 1, 2, 6, 8, 5, 1, 9, 6, 7, 6, 1, 0, 0, 1, 8, 5, 8, 4, 5, 3, 0, 3, 5, 7, 

952


953


954


955


956


957


958


959


960


961


[0, 0, 3, 4, 5, 9, 1, 3, 6, 9, 0, 5, 1, 3, 6, 4, 4, 3, 7, 1, 8, 9, 0, 7, 4, 4, 2, 8, 2, 6, 5, 0, 1, 0, 9, 0, 3, 8, 6, 9, 8, 6, 3, 3, 8, 2, 8, 3, 2, 8, 1, 5, 0, 6, 2, 9, 4, 2, 5, 5, 0, 9, 2, 5, 8, 6, 9, 8, 0, 1, 4, 1, 4, 6, 5, 6, 0, 3, 6, 0, 1, 6, 9, 0, 3, 9, 5, 2, 1, 4, 0, 5, 4, 3, 2, 5, 6, 4, 7, 9, 0, 9, 0, 6, 6, 0, 7, 8, 2, 3, 8, 0, 0, 1, 3, 6, 2, 2, 2, 4, 6, 1, 0, 8, 2, 3, 6, 5, 1, 1, 0, 0, 4, 6, 1, 3, 5, 3, 7, 8, 7, 3, 8, 5, 9, 6, 2, 5, 1, 8, 8, 5, 8, 7, 4, 0, 4, 6, 1, 2, 0, 8, 5, 6, 1, 2, 0, 1, 9, 8, 7, 1, 1, 1, 8, 5, 1, 6, 2, 1, 9, 9, 0, 2, 4, 3, 2, 3, 5, 3, 2, 2, 4, 7, 9, 4, 0, 2, 8, 9, 6, 7, 0, 2, 3, 3, 2, 1, 4, 7, 8, 5, 1, 3, 9, 9, 5, 1, 3, 2, 2, 7, 6, 1, 8, 0, 1, 4, 2, 0, 6, 7, 9, 7, 2, 6, 7, 5, 3, 0, 2, 7, 3, 3, 0, 5, 3, 9, 9, 6, 1, 2, 4, 1, 9, 9, 1, 1, 5, 6, 9, 8, 9, 6, 6, 5, 5, 8, 5, 0, 3, 1, 5, 1, 9, 9, 7, 4, 2, 5, 6, 2, 6, 2, 0, 6, 8, 1, 8, 5, 5, 4, 0, 6, 0, 4, 8, 2, 5, 2, 2, 8, 5, 2, 5, 7, 7, 9, 3, 5, 3, 1, 5, 

982


983


984


985


986


987


988


989


990


991


[1, 0, 8, 4, 9, 4, 1, 7, 2, 1, 0, 7, 3, 8, 6, 2, 7, 0, 1, 6, 4, 2, 2, 2, 4, 0, 1, 6, 7, 8, 9, 9, 1, 9, 9, 0, 6, 6, 6, 1, 7, 9, 2, 1, 3, 1, 7, 3, 1, 0, 1, 1, 7, 6, 1, 4, 6, 9, 2, 9, 3, 8, 0, 5, 1, 5, 9, 6, 3, 9, 4, 1, 4, 5, 9, 9, 0, 1, 6, 0, 0, 6, 1, 7, 2, 1, 5, 4, 9, 0, 6, 5, 4, 1, 9, 0, 4, 8, 9, 9, 2, 0, 8, 1, 1, 9, 9, 8, 9, 3, 7, 5, 0, 1, 9, 8, 5, 2, 4, 1, 9, 3, 5, 5, 6, 2, 6, 7, 5, 2, 9, 1, 1, 6, 3, 3, 5, 3, 7, 0, 1, 8, 2, 4, 9, 5, 0, 2, 6, 8, 0, 7, 5, 4, 4, 0, 3, 4, 6, 9, 7, 4, 5, 1, 1, 9, 9, 1, 9, 4, 8, 0, 7, 2, 1, 6, 2, 6, 1, 5, 2, 3, 7, 9, 9, 3, 9, 7, 7, 6, 2, 1, 0, 8, 6, 6, 9, 5, 6, 5, 0, 1, 2, 8, 9, 5, 1, 5, 6, 7, 8, 2, 6, 9, 8, 9, 8, 2, 3, 2, 2, 3, 6, 1, 3, 0, 0, 9, 0, 0, 8, 5, 6, 7, 9, 7, 8, 7, 9, 3, 2, 3, 2, 0, 3, 1, 2, 6, 5, 1, 1, 8, 5, 3, 7, 6, 0, 1, 6, 0, 9, 1, 7, 6, 0, 1, 0, 1, 8, 0, 1, 8, 4, 3, 4, 9, 8, 2, 5, 7, 2, 5, 1, 6, 3, 6, 6, 1, 0, 1, 6, 5, 0, 1, 0, 8, 6, 1, 6, 5, 9, 0, 5, 8, 7, 1, 7, 9, 3, 3, 8, 1, 0, 

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')